# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('previsao_de_renda.csv')

df = df.dropna()

df = pd.get_dummies(df, columns=['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], drop_first=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     12427 non-null  int64  
 1   data_ref                       12427 non-null  object 
 2   id_cliente                     12427 non-null  int64  
 3   posse_de_veiculo               12427 non-null  bool   
 4   posse_de_imovel                12427 non-null  bool   
 5   qtd_filhos                     12427 non-null  int64  
 6   idade                          12427 non-null  int64  
 7   tempo_emprego                  12427 non-null  float64
 8   qt_pessoas_residencia          12427 non-null  float64
 9   renda                          12427 non-null  float64
 10  sexo_M                         12427 non-null  uint8  
 11  tipo_renda_Bolsista            12427 non-null  uint8  
 12  tipo_renda_Empresário          12427 non-null 

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [15]:
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline

In [16]:
df_2 = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])

df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   posse_de_veiculo               12427 non-null  bool   
 1   posse_de_imovel                12427 non-null  bool   
 2   qtd_filhos                     12427 non-null  int64  
 3   idade                          12427 non-null  int64  
 4   tempo_emprego                  12427 non-null  float64
 5   qt_pessoas_residencia          12427 non-null  float64
 6   renda                          12427 non-null  float64
 7   sexo_M                         12427 non-null  uint8  
 8   tipo_renda_Bolsista            12427 non-null  uint8  
 9   tipo_renda_Empresário          12427 non-null  uint8  
 10  tipo_renda_Pensionista         12427 non-null  uint8  
 11  tipo_renda_Servidor público    12427 non-null  uint8  
 12  educacao_Pós graduação         12427 non-null 

In [17]:
X = df_2.drop(columns=['renda'])
y = df_2['renda']

# Dividir a base em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
# Lista de valores alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Dicionário para armazenar os resultados de R²
r2_ridge = {}

# Ajustar modelos Ridge para diferentes valores de alpha
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_ridge[alpha] = r2
    print(f'Alpha: {alpha}, R²: {r2}')

# Encontrar o melhor modelo com base no R²
melhor_alpha = max(r2_ridge, key=r2_ridge.get)
melhor_r2 = r2_ridge[melhor_alpha]

print(f'\nMelhor modelo: Alpha = {melhor_alpha}, R² = {melhor_r2}')

Alpha: 0, R²: 0.29796640176910294
Alpha: 0.001, R²: 0.29796646662207804
Alpha: 0.005, R²: 0.2979667258970613
Alpha: 0.01, R²: 0.2979670496832443
Alpha: 0.05, R²: 0.2979696277710634
Alpha: 0.1, R²: 0.29797282035425066

Melhor modelo: Alpha = 0.1, R² = 0.29797282035425066


In [19]:
# Dicionário para armazenar os resultados de R²
r2_lasso = {}

# Ajustar modelos Lasso para diferentes valores de alpha
for alpha in alphas:
    lasso = Lasso(alpha=alpha,  max_iter=10000)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_lasso[alpha] = r2
    print(f'Alpha: {alpha}, R²: {r2}')

# Encontrar o melhor modelo com base no R²
melhor_alpha = max(r2_lasso, key=r2_lasso.get)
melhor_r2 = r2_lasso[melhor_alpha]

print(f'\nMelhor modelo: Alpha = {melhor_alpha}, R² = {melhor_r2}')

C:\Users\BrunodaCunhaPeixoto\AppData\Local\Temp\ipykernel_11136\583919730.py:7: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(X_train, y_train)
C:\Users\BrunodaCunhaPeixoto\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\BrunodaCunhaPeixoto\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.036e+11, tolerance: 8.060e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_

Alpha: 0, R²: 0.2979664017691016
Alpha: 0.001, R²: 0.2979668708425244
Alpha: 0.005, R²: 0.29796874449258226
Alpha: 0.01, R²: 0.2979710806056549
Alpha: 0.05, R²: 0.2979895317988459
Alpha: 0.1, R²: 0.29801200134535355

Melhor modelo: Alpha = 0.1, R² = 0.29801200134535355


In [23]:
# Padronizar os dados (opcional, mas recomendado)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ajustar um modelo de regressão linear inicial
model = LinearRegression()

# Usar RFE para seleção de características
rfe = RFE(model, n_features_to_select=1)
rfe.fit(X_train_scaled, y_train)

# Selecionar as melhores características
selected_features = rfe.support_

# Ajustar o modelo com as melhores características
model.fit(X_train_scaled[:, selected_features], y_train)

# Avaliar o modelo no conjunto de teste
y_pred = model.predict(X_test_scaled[:, selected_features])
r2 = r2_score(y_test, y_pred)

print(f'Melhor modelo stepwise R²: {r2}')

# Obter os nomes das características selecionadas
selected_feature_names = X.columns[selected_features]
print(f'Características selecionadas: {selected_feature_names}')

Melhor modelo stepwise R²: 0.1595425826564727
Características selecionadas: Index(['tempo_emprego'], dtype='object')


Dentre as 3 opções, o método que apresentou o melhor $R^2$ foi o Lasso. Para esse caso específico, o Lasso parece ser o modelo mais indicado, porém obteve resultado bastante parecido com Ridge. O Stepwise, por sua vez, apresentou um $R^2$ bem menor do que as outras alternativas.

In [26]:
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures
# Criação de novas variáveis
df['log_tempo_emprego'] = np.log1p(df['tempo_emprego'])

# Transformar a variável dependente para o log da renda
df['log_renda'] = np.log1p(df['renda'])

# Separar as variáveis independentes (X) e dependentes (y)
X = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente', 'renda', 'log_renda'])
y = df['log_renda']

# Dividir a base em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Padronizar os dados - Sem essa padronização, estava dando valores de R2 muito negativos.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Regularização Ridge e Lasso com validação cruzada
alphas = [0.001, 0.005, 0.01, 0.05, 0.1]

ridge_cv = RidgeCV(alphas=alphas, cv=5).fit(X_train_scaled, y_train)
lasso_cv = LassoCV(alphas=alphas, cv=5).fit(X_train_scaled, y_train)

ridge_cv_r2 = r2_score(y_test, ridge_cv.predict(X_test_scaled))
lasso_cv_r2 = r2_score(y_test, lasso_cv.predict(X_test_scaled))

print(f'RidgeCV R²: {ridge_cv_r2}')
print(f'LassoCV R²: {lasso_cv_r2}')

# Transformações polinomiais
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Ajuste do modelo com transformações polinomiais
model_poly = LinearRegression().fit(X_train_poly, y_train)
poly_r2 = r2_score(y_test, model_poly.predict(X_test_poly))

print(f'Polynomial Features R²: {poly_r2}')

RidgeCV R²: 0.36709532516468846
LassoCV R²: 0.3677380764325311
Polynomial Features R²: -1.1832134016615758e+21


Com as transformações realizadas (log da renda e log do tempo emprego), obtivemos um valor de $R^2$ maior que os vistos anteriormente, sendo o valor do Lasso ligeiramente maior que o Ridge. Na hora de transformar em um polinômio, no entanto, o valor do $R^2$ ficou bastante descalibrado, com um valor muito grande e negativo, o que indica um possível overfitting do modelo.

In [28]:
from sklearn.tree import DecisionTreeRegressor
# Ajustar a árvore de regressão
X = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente', 'renda', 'log_renda'])
y = df['log_renda']

tree_regressor = DecisionTreeRegressor(random_state=42)
tree_regressor.fit(X_train_scaled, y_train)

# Previsões e cálculo do R²
y_train_pred = tree_regressor.predict(X_train_scaled)
y_test_pred = tree_regressor.predict(X_test_scaled)

tree_train_r2 = r2_score(y_train, y_train_pred)
tree_test_r2 = r2_score(y_test, y_test_pred)

print(f'Tree Regression Train R²: {tree_train_r2}')
print(f'Tree Regression Test R²: {tree_test_r2}')

Tree Regression Train R²: 0.7793849578261296
Tree Regression Test R²: 0.29137664738855806


Na base de treinamento, a árvore atingiu um $R^2$ bastante elevado, de quase 80%. No teste, no entanto, o valor caiu para 29%, abaixo dos valores obtidos com Ridge e Lasso após as transformações.